In [145]:
!unzip train.csv.zip

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.


In [146]:
!unzip test.csv.zip

unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.


In [147]:
!pip install unidecode

In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from unidecode import unidecode
import re

In [149]:
train = pd.read_csv('train.csv')
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [150]:
train.value_counts()

,,,,,,,,,,,,count
ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,
1,20183,HOTELERIA Y TURISMO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria completa,Secundaria (Bachillerato) incompleta,No,medio-alto,1
659298,20195,ADMINISTRACION EN FINANZAS Y NEGOCIOS INTERNACIONALES,CORDOBA,Entre 500 mil y menos de 1 millón,0,Estrato 1,Si,Secundaria (Bachillerato) completa,Primaria completa,No,medio-alto,1
659281,20195,ADMINISTRACION DE EMPRESAS TURISTICAS,ANTIOQUIA,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,medio-bajo,1
659282,20195,PSICOLOGÍA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Postgrado,Postgrado,No,alto,1
659286,20195,PSICOLOGIA,SUCRE,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 1,Si,Primaria incompleta,Técnica o tecnológica completa,Si,bajo,1
...,...,...,...,...,...,...,...,...,...,...,...,...
329914,20183,DERECHO,ANTIOQUIA,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,Si,alto,1
329915,20183,"LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTES",ANTIOQUIA,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 1,No,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo,1
329916,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Primaria incompleta,Si,bajo,1


# Overview


Para la limpieza del archivo train.csv, se realizarán las siguientes operaciones:


1.   Estandarización de datos:
      *   ESTU_PRGM_ACADEMICO

2.   Relleno de datos faltantes:
      *   **FAMI_ESTRATOVIVIENDA:** se rellenan los vacíos con el estrato que tiene más repeticiones entre todo el dataset, dado que todo estudiante debe pertenecer a uno.
      *   **FAMI_TIENEINTERNET:** asumimos un espacio vacío como un "No".
      *   **FAMI_EDUCACIONPADRE:** asumimos un espacio vacío como un "No aplica" y se suma a las filas que tienen "No sabe" como entrada.
      *   **FAMI_EDUCACIONMADRE:** asumimos un espacio vacío como un "No aplica".
      *   **ESTU_VALORMATRICULAUNIVERSIDAD:** asumimos un espacio vacío como un "No pagó matrícula".
      *   **ESTU_HORASSEMANATRABAJA:** asumimos un espacio vacío como un "No trabaja"
      *   **ESTU_PAGOMATRICULAPROPIO:** asumimos un espacio vacío como un "No".


3.   Sustituimos los valores "Sí" / "No" por "1" / "0" respectivamente:
      *   FAMI_TIENEINTERNET
      *   ESTU_PAGOMATRICULAPROPIO
      *   FAMI_TIENEINTERNET


4.   Usamos One Hot para categorizar las siguientes columnas:
      *   FAMI_ESTRATOVIVIENDA
      *   FAMI_EDUCACIONPADRE
      *   ESTU_HORASSEMANATRABAJA
      *   ESTU_VALORMATRICULAUNIVERSIDAD
      *   ESTU_PRGM_ACADEMICO
      *   ESTU_PRGM_DEPARTAMENTO



































# **1. Estandarización de datos**

*Columna ESTU_PRGM_ACADEMICO*

In [209]:
train['ESTU_PRGM_ACADEMICO'] = (train['ESTU_PRGM_ACADEMICO']
                                 .str.replace(r'^PROFESIONAL EN\s+', '', regex=True)
                                 .str.replace(r'\s+', ' ', regex=True)
                                 .str.upper()
                                 .apply(unidecode)
                               )

reemplazos = {
    r'INGENIE?RIA': 'INGENIERIA',
    r'EDUCACIO?N\b': 'EDUCACION',
    r'RECREACIO?N\b': 'RECREACION',
    r'FISICA\b': 'FISICA',
    r'ESCENICAS\b': 'ESCENICAS',
    r'MATEMATICAS\b': 'MATEMATICAS',
    r'FILOSOFIA\b': 'FILOSOFIA',
    r'DISEÑO\b': 'DISEÑO',
    r'COMUNICACIO?N\b': 'COMUNICACION',
    r'GRAFICA\b': 'GRAFICA',
    r'ELECTRICA\b': 'ELECTRICA',
    r'QUIMICA\b': 'QUIMICA',
    r'FARMACEUTICA\b': 'FARMACEUTICA',
    r'ADMINISTRACIO?N\b': 'ADMINISTRACION',
    r'PUBLICA\b': 'PUBLICA',
    r'INGLES\b': 'INGLES',
    r'ENFASIS\b': 'ENFASIS',
    r'GESTIO?N\b': 'GESTION',
    r'MUSICA\b': 'MUSICA',
    r'ECONOMIA\b': 'ECONOMIA',
    r'GASTRONOMIA\b': 'GASTRONOMIA',
    r'BASICA\b': 'BASICA',
    r'GEOLOGIA\b': 'GEOLOGIA',
    r'PEDAGOGIA\b': 'PEDAGOGIA',
    r'ARTISTICA\b': 'ARTISTICA',
    r'LOGISTICA\b': 'LOGISTICA',
}

for pattern, replacement in reemplazos.items():
    train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(pattern, replacement, regex=True)


In [213]:
reemplazos = {
    r'.*INGENIERIA.*': 'INGENIERIA',
    r'ADMINISTRACION.*': 'ADMINISTRACION',
    r'QUIMICA.*': 'QUIMICA',
    r'DISEÑO.*': 'DISEÑO',
    r'DISENO.*': 'DISEÑO',
    r'.*ARTE.*': 'ARTES',
    r'LICENCIATURA.*': 'LICENCIATURA',
    r'.*DEPORT.*': 'DEPORTE',
    r'CIENCIA.*': 'CIENCIAS',
    r'COMUNICACION.*': 'COMUNICACION',
    r'GESTION.*': 'GESTION',
    r'FILOSOFIA.*': 'FILOSOFIA',
    r'NEGOCIOS.*': 'NEGOCIOS',
    r'.*MERCAD.*': 'MERCADEO',
    r'CINE.*': 'CINE',
    r'.*AGRO.*': 'AGRONOMIA',
    r'HOTELERIA.*': 'HOTELERIA',
    r'GEREN.*': 'GERENCIA',
    r'ECONO.*': 'ECONOMIA',
    r'.*QUIRURGICA.*': 'MEDICINA',
    r'.*LITERA.*': 'LITERATURA',
    r'.*PSICOLO.*' : 'PSICOLOGIA',
    r'.*BIOLOGIA.*' : 'BIOLOGIA',
    r'MATEMATICAS.*': 'MATEMATICAS',
    r'ADMIN.*': 'ADMINISTRACION',
    r'ESTUDIOS.*': 'ESTUDIOS',
    r'FINANZAS.*': 'FINANZAS',
    r'CONSTRUCCION.*': 'CONSTRUCCION',
    r'DIRECCION.*': 'DIRECCION',
    r'RELACIONES.*': 'RELACIONES',
    r'.*FILOL.*': 'FILOLOGIA',
    r'COMERCIO.*': 'COMERCIO',
    r'PUBLICI.*': 'PUBLICIDAD',
    r'CONTADURIA.*': 'CONTADURIA',
    r'GEO.*': 'GEOGRAFIA',
    r'MARK.*': 'MERCADEO',
    r'.*MUSICA.*': 'MUSICA',
    r'.*GASTR.*': 'GASTRONOMIA',
    r'PROFESIONAL.*': 'PROFESIONAL ',
    r'LENG.*': 'LENGUAS',
    r'.*ZOO.*': 'ZOOLOGIA',
    r'.*VETE.*':'VETERINARIA',
    r'HISTORIA.*': 'HISTORIA',
    r'\?': '',
}

for pattern, replacement in reemplazos.items():
    train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(pattern, replacement, regex=True)

In [228]:
reemplazos = {
    'ANTIOQUIA': 'ANDINA',
    'BOYACA': 'ANDINA',
    'BOGOTÁ': 'ANDINA',
    'CALDAS': 'ANDINA',
    'CUNDINAMARCA': 'ANDINA',
    'HUILA': 'ANDINA',
    'NORTE SANTANDER': 'ANDINA',
    'QUINDIO': 'ANDINA',
    'RISARALDA': 'ANDINA',
    'SANTANDER': 'ANDINA',
    'TOLIMA': 'ANDINA',
    'AMAZONAS': 'AMAZONA',
    'CAQUETA': 'AMAZONA',
    'VAUPES': 'AMAZONA',
    'GUAVIARE': 'AMAZONA',
    'PUTUMAYO': 'AMAZONA',
    'VALLE': 'PACIFICA',
    'CHOCO': 'PACIFICA',
    'CAUCA': 'PACIFICA',
    'NARIÑO': 'PACIFICA',
    'ATLANTICO': 'CARIBE',
    'BOLIVAR': 'CARIBE',
    'CESAR': 'CARIBE',
    'CORDOBA': 'CARIBE',
    'LA GUAJIRA': 'CARIBE',
    'MAGDALENA': 'CARIBE',
    'SUCRE': 'CARIBE',
    'SAN ANDRES': 'CARIBE',
    'ARAUCA': 'ORINOQUIA',
    'CASANARE': 'ORINOQUIA',
    'META': 'ORINOQUIA',
    'VICHADA': 'ORINOQUIA'
}

for old_value, new_value in reemplazos.items():
    train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(old_value, new_value)


# **2. Relleno de datos faltantes**

Columna *FAMI_ESTRATOVIVIENDA*

In [153]:
train['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [154]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [155]:
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [156]:
train['FAMI_ESTRATOVIVIENDA'].isna().sum()

0

In [157]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,264808
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


*Columna FAMI_TIENEINTERNET*

In [158]:
train['FAMI_TIENEINTERNET'].unique()

array(['Si', 'No', nan], dtype=object)

In [159]:
train['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,592514
No,73357


In [160]:
train['FAMI_TIENEINTERNET'].isna().sum()

26629

In [161]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].fillna("No")

In [162]:
train['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,592514
No,99986


*Columna FAMI_EDUCACIONPADRE*


In [163]:
train['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [164]:
train['FAMI_EDUCACIONPADRE'].isna().sum()

23178

In [165]:
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].replace(r'Ninguno', 'No Aplica')
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].replace(r'No sabe', 'No Aplica')

In [166]:
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [167]:
train['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,128289
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
No Aplica,71007
Técnica o tecnológica completa,62995
Primaria completa,55958
Postgrado,44169
Educación profesional incompleta,27084


*Columna FAMI_EDUCACIONMADRE*

In [244]:
train['FAMI_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No Aplica', 'No sabe'], dtype=object)

In [245]:
train['FAMI_EDUCACIONMADRE'].isna().sum()

23664

In [246]:
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].replace(r'Ninguno', 'No Aplica')
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].replace(r'No sabe', 'No Aplica')

In [247]:
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [248]:
train['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
No Aplica,43082
Técnica o tecnológica incompleta,27533


*Columna ESTU_VALORMATRICULAUNIVERSIDAD*

In [173]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [174]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

6287

In [175]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("No pagó matrícula")

In [176]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,25815


*Columna ESTU_HORASSEMANATRABAJA*

In [177]:
train['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [178]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [179]:
train['ESTU_HORASSEMANATRABAJA'].isna().sum()

30857

In [180]:
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [181]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,147407
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


*Columna ESTU_PAGOMATRICULAPROPIO*

In [182]:
train['ESTU_PAGOMATRICULAPROPIO'].unique()

array(['No', 'Si', nan], dtype=object)

In [183]:
train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

6498

In [184]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna('No')

In [185]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,388699
Si,303801


# **3. Sustituimos los valores "Sí" / "No" por "1" / "0" respectivamente**

*Columna FAMI_TIENEINTERNET*



In [188]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'Si', 1.0)
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'No', 0.0)
train['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
1.0,592514
0.0,99986


*Columna ESTU_PAGOMATRICULAPROPIO*


In [189]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.0)
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.0)
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0.0,388699
1.0,303801


*Columna FAMI_TIENEINTERNET*

In [190]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'Si', 1.0)
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'No', 0.0)
train['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
1.0,592514
0.0,99986


#**4. Usamos One Hot para categorizar las columnas**

*Columna FAMI_ESTRATOVIVIENDA*


In [191]:
train['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       'Estrato 6', 'Sin Estrato'], dtype=object)

In [192]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']
enc = OrdinalEncoder(categories = [estratos])
train['FAMI_ESTRATOVIVIENDA'] = enc.fit_transform(train[['FAMI_ESTRATOVIVIENDA']])

In [193]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
2.0,264808
3.0,210685
1.0,111991
4.0,65514
5.0,23608
6.0,12605
0.0,3289


In [234]:
scaler = MinMaxScaler()
train['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(train[['FAMI_ESTRATOVIVIENDA']])
train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
0.333333,264808
0.500000,210685
0.166667,111991
0.666667,65514
0.833333,23608
1.000000,12605
0.000000,3289


*Columna FAMI_EDUCACIONPADRE*


In [216]:
train['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
4.0,128289
1.0,125675
8.0,83117
3.0,71654
0.0,71007
6.0,62995
2.0,55958
9.0,44169
7.0,27084


In [195]:
educacionPadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']
enc = OrdinalEncoder(categories = [educacionPadre])
train['FAMI_EDUCACIONPADRE'] = enc.fit_transform(train[['FAMI_EDUCACIONPADRE']])

In [196]:
train['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
4.0,128289
1.0,125675
8.0,83117
3.0,71654
0.0,71007
6.0,62995
2.0,55958
9.0,44169
7.0,27084


In [235]:
scaler = MinMaxScaler()
train['FAMI_EDUCACIONPADRE'] = scaler.fit_transform(train[['FAMI_EDUCACIONPADRE']])
train['FAMI_EDUCACIONPADRE'].value_counts()

,count
FAMI_EDUCACIONPADRE,
0.444444,128289
0.111111,125675
0.888889,83117
0.333333,71654
0.000000,71007
0.666667,62995
0.222222,55958
1.000000,44169
0.777778,27084


In [ ]:
educacionPadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']
enc = OrdinalEncoder(categories = [educacionPadre])
train['FAMI_EDUCACIONMADRE'] = enc.fit_transform(train[['FAMI_EDUCACIONMADRE']])
scaler = MinMaxScaler()
train['FAMI_EDUCACIONMADRE'] = scaler.fit_transform(train[['FAMI_EDUCACIONMADRE']])
train['FAMI_EDUCACIONMADRE'].value_counts()

*Columna ESTU_HORASSEMANATRABAJA*


In [197]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,147407
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [198]:
horasTrabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']
enc = OrdinalEncoder(categories = [horasTrabajo])
train['ESTU_HORASSEMANATRABAJA'] = enc.fit_transform(train[['ESTU_HORASSEMANATRABAJA']])

In [199]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
4.0,249352
0.0,147407
2.0,115857
3.0,92693
1.0,87191


In [236]:
scaler = MinMaxScaler()
train['ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(train[['ESTU_HORASSEMANATRABAJA']])
train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
1.00,249352
0.00,147407
0.50,115857
0.75,92693
0.25,87191


*Columna ESTU_VALORMATRICULAUNIVERSIDAD*


In [200]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,25815


In [201]:
matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']
enc = OrdinalEncoder(categories = [matricula])
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = enc.fit_transform(train[['ESTU_VALORMATRICULAUNIVERSIDAD']])

In [202]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
3.0,204048
4.0,127430
1.0,80263
2.0,78704
5.0,69736
7.0,68014
6.0,38490
0.0,25815


In [237]:
scaler = MinMaxScaler()
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(train[['ESTU_VALORMATRICULAUNIVERSIDAD']])
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
0.428571,204048
0.571429,127430
0.142857,80263
0.285714,78704
0.714286,69736
1.000000,68014
0.857143,38490
0.000000,25815


*Columna ESTU_PRGM_ACADEMICO*


In [217]:
train["ESTU_PRGM_ACADEMICO"].value_counts()

,count
ESTU_PRGM_ACADEMICO,
INGENIERIA,148273
ADMINISTRACION,127997
LICENCIATURA,65084
DERECHO,53244
CONTADURIA,53066
...,...
NARRATIVAS DIGITALES,7
COMPUTACION CIENTIFICA,5
OCEANOGRAFIA,4


In [226]:
totalProgramas = train["ESTU_PRGM_ACADEMICO"].value_counts().index.tolist()
enc = OrdinalEncoder(categories = [totalProgramas])
train["ESTU_PRGM_ACADEMICO"] = enc.fit_transform(train[["ESTU_PRGM_ACADEMICO"]])

In [227]:
train["ESTU_PRGM_ACADEMICO"].value_counts()

,count
ESTU_PRGM_ACADEMICO,
0.0,148273
1.0,127997
2.0,65084
3.0,53244
4.0,53066
...,...
117.0,7
118.0,5
119.0,4


In [238]:
scaler = MinMaxScaler()
train['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(train[['ESTU_PRGM_ACADEMICO']])
train['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
0.000000,148273
0.008264,127997
0.016529,65084
0.024793,53244
0.033058,53066
...,...
0.966942,7
0.975207,5
0.983471,4


*Columna ESTU_PRGM_DEPARTAMENTO*

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['ANDINA', 'CARIBE', 'AMAZONA', 'PACIFICA', 'ORINOQUIA'],
      dtype=object)

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].isna().sum()

0

In [230]:
totalDepartamento = train['ESTU_PRGM_DEPARTAMENTO'].value_counts().index.tolist()
enc = OrdinalEncoder(categories = [totalDepartamento])
train['ESTU_PRGM_DEPARTAMENTO'] = enc.fit_transform(train[['ESTU_PRGM_DEPARTAMENTO']])

In [231]:
train['ESTU_PRGM_DEPARTAMENTO'].value_counts()

,count
ESTU_PRGM_DEPARTAMENTO,
0.0,499712
1.0,105090
2.0,73802
3.0,10351
4.0,3545


In [239]:
scaler = MinMaxScaler()
train['ESTU_PRGM_DEPARTAMENTO'] = scaler.fit_transform(train[['ESTU_PRGM_DEPARTAMENTO']])
train['ESTU_PRGM_DEPARTAMENTO'].value_counts()

,count
ESTU_PRGM_DEPARTAMENTO,
0.00,499712
0.25,105090
0.50,73802
0.75,10351
1.00,3545


In [254]:
totalDepartamento = train['RENDIMIENTO_GLOBAL'].value_counts().index.tolist()
enc = OrdinalEncoder(categories = [totalDepartamento])
train['RENDIMIENTO_GLOBAL'] = enc.fit_transform(train[['RENDIMIENTO_GLOBAL']])

In [255]:
scaler = MinMaxScaler()
train['RENDIMIENTO_GLOBAL'] = scaler.fit_transform(train[['RENDIMIENTO_GLOBAL']])
train['RENDIMIENTO_GLOBAL'].value_counts()

,count
RENDIMIENTO_GLOBAL,
0.000000,175619
0.333333,172987
0.666667,172275
1.000000,171619


In [241]:
train.drop(columns=['ID'], inplace=True)
train.drop(columns=['PERIODO'], inplace=True)

In [256]:
train.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,0.074380,0.00,0.857143,0.25,0.500000,1.0,0.555556,9.0,0.0,1.000000
1,0.024793,0.25,0.571429,0.00,0.500000,0.0,0.666667,5.0,0.0,0.333333
2,0.115702,0.00,0.571429,1.00,0.500000,1.0,0.444444,4.0,0.0,0.333333
3,0.008264,0.00,0.714286,0.00,0.666667,1.0,0.000000,4.0,0.0,0.000000
4,0.041322,0.00,0.571429,0.75,0.500000,1.0,0.222222,2.0,0.0,0.666667
